In [1]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import time
from io import StringIO
import matplotlib.pyplot as plt
from deuces import Card, Evaluator
from multiprocessing import Pool, cpu_count

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError


from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

sys.path.append('../src')
from calc_preflop_rank import evaluate2
from get_stats import get_player_stats
from monte_carlo import run_monte_carlo

data = pd.read_csv('../data/pluribus_parsed.csv')
data2 = pd.read_csv('../data/archive_parsed.csv')

#data.head()

In [2]:
def find_player(players, seat):
    if 0 <= seat-1 < len(players):
        return players[seat-1]
    return None

def convert_to_deuces_format(card_str):
    return Card.new(card_str)

def find_seat(players, player):
    for i in range(6):
        if players[i] == player:
            return i
    return None

def encode_actions(seat, action_str):
    action_map = {
        'posts small': 0,
        'posts big': 1,
        'folds': 2,
        'calls': 3,
        'raises': 4,
        'bets': 5,
        'checks': 6,
        'uncalled bet': 7,
        'shows': 8,
        'collected': 9
    }
    
    # Split the action string into parts
    parts = action_str.split()
    
    # Extract the action
    if len(parts) > 2 and ' '.join(parts[1:3]) in action_map:
        action = ' '.join(parts[1:3])        
    else:
        action = parts[1]
    
    # Get the action integer
    action_int = action_map.get(action,-1) #default to -1 if action not found
    
    # Extract the value if applicable
    if action in ['raises', 'bets', 'collected', 'posts small blind', 'posts big blind']:
        value = int(float(parts[2])) if action == 'raises' or action == 'collected' else int(parts[-1])
    else:
        value = 0
    
    return [seat, action_int, value]

print(encode_actions(4, "MrBlue: raises 50 to 700"))  # Output: [4, 4, 50]
print(encode_actions(2, "MrPink: posts small blind 10"))  # Output: [2, 0, 10]
print(encode_actions(3, "MrGreen: posts big blind 20"))  # Output: [3, 1, 20]
print(encode_actions(1, "MrYellow: checks"))  # Output: [1, 6, 0]
print(encode_actions(5, "Pluribus: collected 7563.0 from pot")) # Output: [5, 9, 7563]
print(encode_actions(2, "Pluribus: shows [4h Ac]")) # Output: [2, 8, 0]

def decode_actions(players, seat, action_val, value):
    reverse_action_map = {
        0: 'posts small',
        1: 'posts big',
        2: 'folds',
        3: 'calls',
        4: 'raises',
        5: 'bets',
        6: 'checks',
        7: 'uncalled bet',
        8: 'shows',
        9: 'collected'
    }
    seat_int = round(seat)
    action_int = round(action_val)
    value_int = round(value)
    # Get the action string from the action integer
    action = reverse_action_map.get(action_int, 'unknown')
    
    # Find the player from the seat
    player = find_player(players, seat_int)
    
    # Construct the action string
    if action in ['raises', 'bets', 'collected']:
        action_str = f"{player}: {action} {value_int}"
    elif action in ['posts small', 'posts big']:
        action_str = f"{player}: {action} blind {value_int}"
    else:
        action_str = f"{player}: {action}"
    
    return action_str

# Example usage
players = ["MrBlonde", "MrWhite", "MrPink", "MrOrange", "Pluribus", "MrBlue"]
encoded_action = [4, 4, 50]  # Example encoded action
decoded_action = decode_actions(players, *encoded_action)
print(decoded_action)  # Output: "Pluribus: raises 50"

[4, 4, 50]
[2, 0, 0]
[3, 1, 0]
[1, 6, 0]
[5, 9, 7563]
[2, 8, 0]
MrOrange: raises 50


In [3]:
'''
Features:
 - player 1-6 stats
 - hole cards
 - community cards
 - preflop hand rating
 - postflop hand rating
'''

evaluator = Evaluator()

def extract_features(row):
    features = []
    flop_cards = str(row['flop_cards']).split() if pd.notna(row['flop_cards']) and row['flop_cards'] else []
    turn_card = [str(row['turn_card'])] if pd.notna(row['turn_card']) and row['turn_card'] else []
    river_card = [str(row['river_card'])] if pd.notna(row['river_card']) and row['river_card'] else []
    community_cards = [convert_to_deuces_format(card) for card in flop_cards + turn_card + river_card]
    hole_cards_dict = dict([player.split(': ') for player in row['hole_cards'].split(', ')])
   
    for i in range(5):
        features.append(community_cards[i] if i < len(community_cards) else 0)
    
    for player, cards in hole_cards_dict.items():
        
        player_hole_cards = [convert_to_deuces_format(card) for card in cards.split()]
        
        for card in player_hole_cards:
            features.append(card)
        
        try:
            for i in range(4):
                features.append(get_player_stats(player)[i])
        except:
            for i in range(4):
                features.append(0)
        features.append(evaluate2(cards))
        if len(community_cards) == 0:
            features.append(0)
        else:
            features.append(evaluator.evaluate(player_hole_cards, community_cards))
    
    return features 
    
X = np.array([extract_features(row) for _, row in data.iterrows()])
print(X)
print(X.shape)

[[ 2.11482900e+06  5.33255000e+05  8.40680300e+06 ...  8.00000000e-02
   3.58066568e-01  2.09500000e+03]
 [ 2.10254100e+06  8.42318700e+06  1.68120550e+07 ...  7.10000000e-01
   4.43237907e-01  6.89100000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -6.20000000e-01
   5.05257333e-01  0.00000000e+00]
 ...
 [ 2.66757000e+05  6.71196470e+07  1.68120550e+07 ...  7.10000000e-01
   3.38641189e-01  2.29200000e+03]
 [ 4.22862500e+06  1.67833830e+07  1.34236965e+08 ...  8.00000000e-02
   3.09988942e-01  2.62600000e+03]
 [ 5.57831000e+05  1.34228773e+08  3.35649570e+07 ... -1.20000000e-01
   5.81757508e-01  3.33800000e+03]]
(10000, 53)


In [4]:
def extract_labels(row):
    labels = []
    
    def parse_actions(actions_str):
        actions = actions_str.split(', ')
        return [encode_actions(find_seat(row['players'].split(', '), action.split(': ')[0]), action) for action in actions]
    
    all_actions = []
    if pd.notna(row['preflop_actions']):
        all_actions.extend(parse_actions(row['preflop_actions']))
    if pd.notna(row['flop_actions']):
        all_actions.extend(parse_actions(row['flop_actions']))
    if pd.notna(row['turn_actions']):
        all_actions.extend(parse_actions(row['turn_actions']))
    if pd.notna(row['river_actions']):
        all_actions.extend(parse_actions(row['river_actions']))
    if pd.notna(row['showdown_actions']):
        all_actions.extend(parse_actions(row['showdown_actions']))
    
    max_actions = 16
    all_actions = all_actions[:max_actions]
    all_actions += [[0,0,0]] * (max_actions - len(all_actions))
    labels.extend(all_actions)
    labels.append(encode_actions(find_seat(row['players'].split(', '), row['winners'].split(': ')[0]), row['winners']))
    
    return labels

y = np.array([extract_labels(row) for _, row in data.iterrows()])
print(y)
print(y.shape)

[[[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   0    7    0]
  [   0    0    0]
  [   0    9  520]]

 [[   0    0    0]
  [   1    1    0]
  [   2    4  110]
  ...
  [   2    7    0]
  [   0    0    0]
  [   2    9  940]]

 [[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   0    0    0]
  [   0    0    0]
  [   5    9  250]]

 ...

 [[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   1    6    0]
  [   3    5 1750]
  [   3    9 2320]]

 [[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   0    5  675]
  [   1    2    0]
  [   0    9  675]]

 [[   0    0    0]
  [   1    1    0]
  [   2    4  125]
  ...
  [   1    6    0]
  [   2    6    0]
  [   2    9  725]]]
(10000, 17, 3)


In [5]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(53,)))

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(hp.Int('units_' + str(i), min_value=64, max_value=256, step=32), activation='relu'))
        model.add(layers.Dropout(hp.Float('dropout_' + str(i), min_value=0.2, max_value=0.5, step=0.1)))

    # Output layer for classification
    model.add(layers.Dense(17 * 3, activation='softmax'))  # Softmax for multi-class classification
    model.add(layers.Reshape((17, 3)))  # Reshape output to match y_train shape

    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop'])
    learning_rate = hp.Float('learning_rate', min_value=.0001, max_value=.01, sampling='log')

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',  # Loss for classification
        metrics=['accuracy']  # Accuracy for evaluation
    )
    return model


# Directory for tuning results
directory = 'my_dir'

def remove_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)

remove_dir(directory)

# Seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_mae',  # Using Mean Absolute Error for tuning
    max_epochs=50,
    hyperband_iterations=2,
    directory=directory,
    project_name='my_project'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_mae',
    patience=5,
    min_delta=.001,
    restore_best_weights=True
)

# Perform hyperparameter search
tuner.search(
    X_train, 
    y_train, 
    epochs=50, 
    validation_split=0.2, 
    callbacks=[early_stopping]
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Save and reload the best model
best_model.save('best_model.keras')
best_model = tf.keras.models.load_model('best_model.keras')

# Evaluate the model
loss, mae = best_model.evaluate(X_test, y_test, verbose=1)
print(f'Loss: {loss}')
print(f'Mean Absolute Error: {mae}')

Trial 2 Complete [00h 00m 01s]

Best val_mae So Far: None
Total elapsed time: 00h 00m 04s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
256               |64                |units_0
0.2               |0.4               |dropout_0
rmsprop           |adam              |optimizer
0.0022412         |0.00015905        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
196/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5674 - loss: nan

Traceback (most recent call last):
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jakep\VscodeProjects\poker-AI\myenv\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fi

In [ ]:
expected_shape = best_model.input_shape[1]

def pad_features(features, max_length):
    if len(features) < max_length:
        features.extend([0] * (max_length - len(features)))
    return features

X_new = np.array([pad_features(extract_features(row), expected_shape) for _, row in data.iterrows() if extract_features(row) is not None])
print(X_new)

y_new = np.array([extract_labels(row) for _, row in data.iterrows()])
print(y_new)



[[ 2.11482900e+06  5.33255000e+05  8.40680300e+06 ...  8.00000000e-02
   3.58066568e-01  5.05600000e+03]
 [ 2.10254100e+06  8.42318700e+06  1.68120550e+07 ...  7.10000000e-01
   4.43237907e-01  6.89100000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -6.20000000e-01
   5.05257333e-01  0.00000000e+00]
 ...
 [ 2.66757000e+05  6.71196470e+07  1.68120550e+07 ...  7.10000000e-01
   3.38641189e-01  5.63500000e+03]
 [ 4.22862500e+06  1.67833830e+07  1.34236965e+08 ...  8.00000000e-02
   3.09988942e-01  6.91700000e+03]
 [ 5.57831000e+05  1.34228773e+08  3.35649570e+07 ... -1.20000000e-01
   5.81757508e-01  6.24700000e+03]]
[[[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   0    7    0]
  [   0    0    0]
  [   0    9  520]]

 [[   0    0    0]
  [   1    1    0]
  [   2    4  110]
  ...
  [   2    7    0]
  [   0    0    0]
  [   2    9  940]]

 [[   0    0    0]
  [   1    1    0]
  [   2    2    0]
  ...
  [   0    0    0]
  [   0    0    0]
  [   5    9  250]]

In [ ]:
# Compile the model with appropriate loss and metrics for classification
best_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for classification
                   metrics=['accuracy'])  # Metrics like 'accuracy' for classification

# Evaluate the model
try:
    # Ensure X_test and y_test have the correct shapes
    loss, accuracy = best_model.evaluate(X_test, y_test, verbose=1)
    print(f'Loss: {loss}')
    print(f'Accuracy: {accuracy}')  # Accuracy metric for classification
except ValueError as e:
    print(f'ValueError during model evaluation: {e}')
except AttributeError as e:
    print(f'AttributeError during model evaluation: {e}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.5694 - loss: nan
Loss: nan
Accuracy: 0.5696765780448914


In [ ]:
#provide a game state
game_state = (pd.read_csv('../data/game_state.csv'))

def predict_game(model, game):
    for _, row in game.iterrows():
        X = np.array([extract_features(row)])
        print("Input shape:", X.shape)
        actions = model.predict(X)
        print("Predicted actions:\n")
        for action in actions[0]:
            print(decode_actions(row['players'].split(', '), *action))
        
predict_game(best_model, game_state)

Input shape: (1, 53)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted actions:



ValueError: cannot convert float NaN to integer